In [ ]:
# ページを訪れる
# htmlデータを更新
# 前回から変更があるかないかをチェックしてフラグを更新
# latestを更新
# countを更新
# 辿り元の記事からの脈絡を見て報酬やバツを与え、リンクのスコアを更新する。
# 次のターゲットを決定する
    # hrefsの中から選ぶ （どうやって選ぶ？ -> スコアからリンクの確率分布を作る）
    # 辿ろうとしてみる
        # 辿れなかったら active: False にして別の hrefs の中からまた選ぶ
# hrefデータも更新
    # activeなものをすべて辿ったら、一度hrefsを更新してみる
    # 過去に存在して今は存在しないリンクはアーカイブする
    # 過去にもあって今もあるリンクは変えない、リンクの数が変わっていたらnumを修正する
    # 過去に無くて今はあるリンクは新しくつくる
# ターゲットが決まって辿ることができそうであれば辿り、 1.countに加算, 2.latestを更新し、 #最初に戻る

# このサイクルをスタート地点に戻るまで繰り返す。
# 順次indexにURLを保存していく
# この1サイクルを1サイクルとしてurlのリストとして保存しておく。
# サイクルのスコアも算出する（1ステップで得られるスコアの総和やチェーンの長さ）
# サイクルのスコアが高ければ、親和性の高い記事の順序だった一群が得られる。
# この出力結果をベースにして、
# この記事群を対象にして、記事のリファクタリングやタグの自動生成といったさらに詳細な分析（ナレッジの継続的洗練）
    # 記事を分割したり”言葉足らずな”記事を書くことや”補足的な”記事を書くことが怖くなくなる。
# 検索入力に対するファジーな提案
# などの機能を作り込むことができる

In [1]:
import requests
from bs4 import BeautifulSoup as bs4

import hashlib
import json, re, collections, dataclasses
from src.crawler import Crawler

In [2]:
domain = "https://math.jp"
lp = "/wiki/%E3%83%A1%E3%82%A4%E3%83%B3%E3%83%9A%E3%83%BC%E3%82%B8"
target = "/wiki/%E6%B8%AC%E5%BA%A6%E3%81%A8%E7%A9%8D%E5%88%861%EF%BC%9A%E6%B8%AC%E5%BA%A6%E8%AB%96%E3%81%AE%E5%9F%BA%E7%A4%8E%E7%94%A8%E8%AA%9E"

res = requests.get(domain)
type(res)
type(res) is requests.models.Response
res.url

'https://math.jp/wiki/%E3%83%A1%E3%82%A4%E3%83%B3%E3%83%9A%E3%83%BC%E3%82%B8'

In [3]:
with open("tmp/index.json", "r") as f:
    s = f.read()
    if s:
        j = json.loads(s)
    else:
        j = []

len(j)

5

In [4]:
crawler = Crawler(domain)
len(crawler.index)
crawler.crawling()

now i'm in https://math.jp/wiki/%E4%BB%A3%E6%95%B0%E7%9A%84%E3%83%88%E3%83%9D%E3%83%AD%E3%82%B8%E3%83%BC
now i'm in https://math.jp/wiki/%E6%9C%80%E8%BF%91%E3%81%AE%E6%9B%B4%E6%96%B0
now i'm in https://math.jp/wiki/%E8%A4%87%E7%B4%A0%E8%A7%A3%E6%9E%90%E3%81%AE%E5%88%9D%E6%AD%A9%23.E5.AE.9A.E7.90.866.1.EF.BC.88.E6.AD.A3.E5.89.87.E9.96.A2.E6.95.B0.E3.81.AE.E8.A7.A3.E6.9E.90.E6.80.A7.EF.BC.89
now i'm in https://math.jp/wiki/%E7%89%B9%E5%88%A5:%E7%89%B9%E5%88%A5%E3%83%9A%E3%83%BC%E3%82%B8%E4%B8%80%E8%A6%A7


In [66]:
domain = "https://math.jp"
lp = "/wiki/%E3%83%A1%E3%82%A4%E3%83%B3%E3%83%9A%E3%83%BC%E3%82%B8"
target = "/wiki/%E6%B8%AC%E5%BA%A6%E3%81%A8%E7%A9%8D%E5%88%861%EF%BC%9A%E6%B8%AC%E5%BA%A6%E8%AB%96%E3%81%AE%E5%9F%BA%E7%A4%8E%E7%94%A8%E8%AA%9E"

res = requests.get(domain+target)
soup = bs4(res.content, "html.parser")

# soup.find_all("a")
hrefs = [ a.get("href") for a in soup.find_all("a") if a.get("href").startswith('/wiki/') ]
hist = collections.Counter(hrefs).most_common()

hrefs = [ {"profile": {"first": "yet", "latest": "yet", "active": True}, "count": 0,  "num": href[1], "score": 0, "url": href[0] } for href in hist ]
data = {
    "url": target,
    "first": "yet",
    "latest": "yet",
    "count": 0,
    "hrefs": hrefs,
    "content": str(soup.select("html")[0]),
    "edited": False,
}

file_name = hashlib.sha1(url_tg.encode('utf-8')).hexdigest()
with open(f"data/html/{file_name}.json", "w") as f:
    json.dump(data, f, indent=2)


# 周遊の Reinforced Train で概念間の繋がりを理解させる

# ソース概観

In [35]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="ja">
 <head>
  <meta charset="utf-8"/>
  <title>
   測度と積分1：測度論の基礎用語 - Mathpedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"ja","wgMonthNames":["","1月","2月","3月","4月","5月","6月","7月","8月","9月","10月","11月","12月"],"wgRequestId":"ZObbkq-L0DnIiPIJZKIr5QAAAIA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"測度と積分1：測度論の基礎用語","wgTitle":"測度と積分1：測度論の基礎用語","wgCurRevisionId":8684,"wgRevisionId":8684,"wgArticleId":355,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgPageContentLanguage":"ja","wgPageContentModel":"wikitext","wgRelevantPageName":"測度と積分1：測度論の基礎用語","wgRelevantArticleId":355,"wgIsProbablyEditable":!1,"wgRelevantPageIsProbablyEditable":!1,"wgRestrictionEdit":[],"w

In [38]:
for string in soup.body.find(id="content").stripped_strings:
    print(repr(string))

'測度と積分1：測度論の基礎用語'
'提供: Mathpedia'
'名前空間'
'ページ'
'議論'
'その他'
'その他'
'言語'
'ページアクション'
'閲覧'
'ソースを表示'
'履歴'
'この章では、可測空間と可測写像について基本的なことを述べる。可測空間とは測度と積分が定義される空間であり、可測写像とは可測空間の構造を保つような可測空間の間の写像である。可測空間と可測写像の関係は、位相空間と連続写像のような関係である。実際、位相空間が位相（開集合族）が付与された空間であり、連続写像は位相空間の間の写像で開集合の逆像が開集合であるようなものであるのに対し、可測空間は $\\sigma$-加法族（可測集合族）が付与された空間であり、可測写像は可測空間の間の写像で可測集合の逆像が可測集合であるようなものである。'
'入門テキスト「測度と積分」'
'測度と積分1：測度論の基礎用語'
'測度と積分2：測度空間上の積分'
'測度と積分3：測度論の基本定理(1)'
'測度と積分4：測度論の基本定理(2)'
'測度と積分5：$L^p$ 空間の完備性と双対性'
'測度と積分6：数え上げ測度と $\\ell^p$ 空間'
'測度と積分7：局所コンパクトHausdorff空間上のRadon測度'
'測度と積分8：Lebesgue測度の基本的性質'
'測度と積分9：Bochner積分'
'目次'
'1'
'1. $\\sigma$-加法族、可測空間、Borel集合族'
'1.1'
'定義1.1（$\\sigma$-加法族、可測空間）'
'1.2'
'命題1.2'
'1.3'
'定義1.3（相対 $\\sigma$-加法族）'
'1.4'
'定義1.4（部分集合族から生成される $\\sigma$-加法族）'
'1.5'
'定義1.5（位相空間のBorel集合族）'
'1.6'
'補題1.6'
'1.7'
'命題1.7（相対位相と相対Borel集合族）'
'2'
'2. 写像の可測性、直積可測空間'
'2.1'
'定義2.1（可測写像（可測関数））'
'2.2'
'定義2.2（Borel写像（Borel関数））'
'2.3'
'命題2.3（写像が可測であるための条件）'
'2.4'
'系2.4（連続写像はBorel写像）'
'2.5'
'定義2.5（直積 $\\sigma$-加

# 保存用データの切り出し方

- 目次には用語が端的にまとまっている。
- $\LaTeX$記法の要素も、正規表現で取り出してくることはできそう